## SQL Queries on Bay Area Bikeshare Dataset

Bikeshare database from September 2015 - August 2016 with four tables: trips, stations, status, and weather. Source in csv form, along with explanatory metadata, at http://www.bayareabikeshare.com/open-data.

### The ID's and durations for all trips of duration greater than 500, ordered by duration.

SELECT

    trip_id,
    CAST(duration as INTEGER) duration
FROM

    trips
WHERE
    
    duration > 500
ORDER BY 2 DESC

### Every column of the stations table for station id 84.

SELECT    
    
    *
FROM    
    
    stations
WHERE    

    station_id = 84

### The min temperatures of all the occurrences of rain in zip 94301.

SELECT   
    
    MinTemperatureF    
FROM

    weather    
WHERE

    ZIP = 94301 AND
    Events like 'Rain'

### What was the hottest day in our data set? Where was that?

SELECT
    
    ZIP,
    MAX(MaxTemperatureF) as max_temp    
FROM

    weather

### How many trips started at each station?

SELECT
    
    start_station,
    COUNT(*) as station_count    
FROM
    
    trips    
GROUP by 1

### What's the shortest trip that happened?

SELECT

    trip_id,
    MIN(CAST(duration as INTEGER)) min_duration    
FROM

    trips

### What is the average trip duration, by end station?

SELECT
    
    end_station,
    AVG(CAST(duration as INTEGER)) avg_duration
FROM
    
    trips    
GROUP BY 1

### What are the three longest trips on rainy days?

WITH

    trip_rain    
AS (
    
    SELECT
        date
    FROM
        weather
    WHERE
        events like 'rain'   
	GROUP BY 1
)

SELECT
    
    trip_id,
    CAST(duration as INTEGER) duration,
    DATE(start_date) trip_date
FROM
    
    trips
JOIN
    
    trip_rain
ON
    
    trip_rain.date = trip_date
ORDER BY 2 DESC

LIMIT 3

### Which station is empty most often?

SELECT
    
    status.station_id,
    stations.name,
    COUNT(CASE WHEN bikes_available = 0 THEN 1 END) as empty_count
FROM

    status
JOIN

    stations
ON

    status.station_id = stations.station_id    
GROUP BY 1

ORDER BY 3 DESC

### Return a list of stations with a count of number of trips starting at that station but ordered by dock count.

SELECT

    trips.start_station,
    COUNT(start_station) trip_count,
    stations.dockcount    
FROM

    trips    
JOIN

    stations    
ON

    trips.start_station = stations.name    
GROUP BY 1

ORDER BY 3 DESC

### What's the length of the longest trip for each day it rains anywhere?

WITH

	event_rain     
AS(

	SELECT 
		date
	FROM 
		weather
	WHERE events like 'Rain'
	GROUP BY 1    
),
	
    rainy_trips 
AS(

	SELECT
		trip_id,
		CAST(duration as INTEGER) duration,
		DATE(start_date) trip_date
	FROM 
		trips
	JOIN 
		event_rain
	ON 
		event_rain.date = trip_date
)

SELECT 

	trip_id,
	trip_date,
	max(duration)
FROM 
	
    rainy_trips    
GROUP BY 2

#### Alternative:

WITH

	event_rain     
AS(

	SELECT 
		date
	FROM 
		weather
	WHERE events like 'Rain'
	GROUP BY 1    
)

SELECT

	trip_id,
	DATE(start_date) trip_date,
	MAX(CAST(duration as INTEGER)) duration    
FROM

	trips    
JOIN

	event_rain    
ON

	event_rain.date = trip_date    
GROUP BY 2